In [ ]:
!pip install pydicom

In [ ]:
!pip install kornia

In [ ]:
!pip install fastai2

In [ ]:
from fastai2.vision.all import *

In [ ]:
#!pip install kornia

In [ ]:
from fastai2.basics import *
from fastai2.medical.imaging import *

Let's take a look at what files we have in the dataset.

In [ ]:
path = Path('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/')

Most lists in fastai v2, including that returned by `Path.ls`, are returned as a [fastai.core.L](http://dev.fast.ai/core.html#L), which has lots of handy methods, such as `attrgot` used here to grab file names.

In [ ]:
path_trn = path/'stage_2_train'
fns_trn = path_trn.ls()
fns_trn[:5].attrgot('name')

In [ ]:
path_tst = path/'stage_2_test'
fns_tst = path_tst.ls()
len(fns_trn),len(fns_tst)

We can grab a file and take a look inside using the `dcmread` method that fastai v2 adds.

In [ ]:
fn = fns_trn[0]
dcm = fn.dcmread()
dcm

# Labels

Before we pull the metadata out of the DIMCOM files, let's process the labels into a convenient format and save it for later. We'll use *feather* format because it's lightning fast!

In [ ]:
def save_lbls():
    path_lbls = path/'stage_2_train.csv'
    lbls = pd.read_csv(path_lbls)
    lbls[["ID","htype"]] = lbls.ID.str.rsplit("_", n=1, expand=True)
    lbls.drop_duplicates(['ID','htype'], inplace=True)
    pvt = lbls.pivot('ID', 'htype', 'Label')
    pvt.reset_index(inplace=True)    
    pvt.to_feather('labels.fth')

In [ ]:
save_lbls()

In [ ]:
df_lbls = pd.read_feather('labels.fth').set_index('ID')
df_lbls.head(8)
len(df_lbls)

In [ ]:
df_samp=pd.read_csv(path/'stage_2_sample_submission.csv')

In [ ]:
"""df_trn=pd.read_csv(path/'stage_2_train.csv')
df_trn[["ID","htype"]] = df_trn.ID.str.rsplit("_", n=1, expand=True)
df_trn.head()  """

In [ ]:
df_samp[["ID","htype"]] = df_samp.ID.str.rsplit("_", n=1, expand=True)
df_samp.drop_duplicates(['ID','htype'], inplace=True)

pvt=df_samp.pivot('ID','htype','Label')
pvt.reset_index(inplace=True)


# DICOM Meta

To turn the DICOM file metadata into a DataFrame we can use the `from_dicoms` function that fastai v2 adds. By passing `px_summ=True` summary statistics of the image pixels (mean/min/max/std) will be added to the DataFrame as well (although it takes much longer if you include this, since the image data has to be uncompressed).

In [ ]:
%time df_tst = pd.DataFrame.from_dicoms(fns_tst, px_summ=True)
df_tst.to_feather('df_tst.fth')
df_tst.head()

In [ ]:
del(df_tst)
gc.collect();

In [ ]:
%time df_trn = pd.DataFrame.from_dicoms(fns_trn, px_summ=True)
df_trn.to_feather('df_trn.fth')

There is one corrupted DICOM in the competition data, so the command above prints out the information about this file.

In [ ]:

from IPython.display import FileLink
FileLink(r'df_trn.fth')